In [36]:
import pandas as pd
import datetime

In [54]:
def data_preprocessing(sample, ticker, base_date):   
    sample['CODE'] = ticker # 종목코드 추가
    sample = sample[sample['Date'] >= base_date][['Date','CODE','Adj Close']].copy() # 기준일자 이후 데이터 사용
    sample.reset_index(inplace= True, drop= True)
    # 기준년월 
    sample['STD_YM'] = sample['Date'].map(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m')) 
    sample['1M_RET'] = 0.0 # 수익률 컬럼
    ym_keys = list(sample['STD_YM'].unique()) # 중복 제거한 기준년월 리스트
    return sample, ym_keys

In [55]:
def create_trade_book(sample, sample_codes):
    book = pd.DataFrame()
    book = sample[sample_codes].copy()
    book['STD_YM'] = book.index.map(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m'))
    for c in sample_codes:
        book['p '+c] = ''
        book['r '+c] = ''
    return book

In [56]:
# 상대모멘텀 tradings
def tradings(book, s_codes):
    std_ym = ''
    buy_phase = False
    # 종목코드별 순회
    for s in s_codes : 
        print(s)
        # 종목코드 인덱스 순회
        for i in book.index:
            # 해당 종목코드 포지션을 잡아준다. 
            if book.loc[i,'p '+s] == '' and book.shift(1).loc[i,'p '+s] == 'ready ' + s:
                std_ym = book.loc[i,'STD_YM']
                buy_phase = True
            # 해당 종목코드에서 신호가 잡혀있으면 매수상태를 유지한다.
            if book.loc[i,'p '+s] == '' and book.loc[i,'STD_YM'] == std_ym and buy_phase == True : 
                book.loc[i,'p '+s] = 'buy ' + s
            
            if book.loc[i,'p '+ s] == '' :
                std_ym = None
                buy_phase = False
    return book

In [57]:
def multi_returns(book, s_codes):
    # 손익 계산
    rtn = 1.0
    buy_dict = {}
    num = len(s_codes)
    sell_dict = {}
    
    for i in book.index:
        for s in s_codes:
            if book.loc[i, 'p ' + s] == 'buy '+ s and \
            book.shift(1).loc[i, 'p '+s] == 'ready '+s and \
            book.shift(2).loc[i, 'p '+s] == '' :     # long 진입
                buy_dict[s] = book.loc[i, s]
#                 print('진입일 : ',i, '종목코드 : ',s ,' long 진입가격 : ', buy_dict[s])
            elif book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i, 'p '+s] == 'buy '+ s:     # long 청산
                sell_dict[s] = book.loc[i, s]
                # 손익 계산
                rtn = (sell_dict[s] / buy_dict[s]) -1
                book.loc[i, 'r '+s] = rtn
                print('개별 청산일 : ',i,' 종목코드 : ', s , 'long 진입가격 : ', buy_dict[s], ' |  long 청산가격 : ',\
                      sell_dict[s],' | return:', round(rtn * 100, 2),'%') # 수익률 계산.
            if book.loc[i, 'p '+ s] == '':     # zero position || long 청산.
                buy_dict[s] = 0.0
                sell_dict[s] = 0.0


    acc_rtn = 1.0        
    for i in book.index:
        rtn  = 0.0
        count = 0
        for s in s_codes:
            if book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i,'p '+ s] == 'buy '+ s: 
                # 청산 수익률계산.
                count += 1
                rtn += book.loc[i, 'r '+s]
        if (rtn != 0.0) & (count != 0) :
            acc_rtn *= (rtn /count )  + 1
            print('누적 청산일 : ',i,'청산 종목수 : ',count, \
                  '청산 수익률 : ',round((rtn /count),4),'누적 수익률 : ' ,round(acc_rtn, 4)) # 수익률 계산.
        book.loc[i,'acc_rtn'] = acc_rtn
    print ('누적 수익률 :', round(acc_rtn, 4))

In [58]:
df = pd.read_csv('./csv/AAPL.csv')
price_df, ym_keys = data_preprocessing(df,'AAPL',base_date='2010-01-02')

In [59]:
price_df.head()

,Date,CODE,Adj Close,STD_YM,1M_RET
0,2010-01-04,AAPL,26.782711,2010-01,0.0
1,2010-01-05,AAPL,26.829010,2010-01,0.0
2,2010-01-06,AAPL,26.402260,2010-01,0.0
3,2010-01-07,AAPL,26.353460,2010-01,0.0
4,2010-01-08,AAPL,26.528664,2010-01,0.0


In [60]:
import os 
import glob
import pandas as pd
import numpy as np
import datetime

#종목 데이터 읽어오기.
files = glob.glob('./csv/*.csv')

# 필요한 데이터 프레임 생성
# Monthly 데이터를 저장하기 위함이다.
month_last_df = pd.DataFrame(columns=['Date','CODE','1M_RET'])
# 종목 데이터 프레임 생성
stock_df = pd.DataFrame(columns =['Date','CODE','Adj Close'])

for file in files:
    """
    데이터 저장 경로에 있는 개별 종목들을 읽어온다.
    """
    if os.path.isdir(file):
        print('%s <DIR> '%file)
    else:
        folder, name = os.path.split(file)
        head, tail = os.path.splitext(name)
        print(file)
        read_df = pd.read_csv(file) # 경로를 읽은 데이터를 하나씩 읽어들인다.
        
        # 1단계. 데이터 가공
        price_df, ym_keys = data_preprocessing(read_df,head,base_date='2010-01-02')
        # 가공한 데이터 붙이기.
        stock_df = stock_df.append(price_df.loc[:,['Date','CODE','Adj Close']],sort=False)
        # 월별 상대모멘텀 계산을 위한 1개월간 수익률 계산
        for ym in ym_keys:
            m_ret = price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],'Adj Close'] \
            / price_df.loc[price_df[price_df['STD_YM'] == ym].index[0],'Adj Close'] 
            price_df.loc[price_df['STD_YM'] == ym, ['1M_RET']] = m_ret
            month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
                                                              ['Date','CODE','1M_RET']])    


./csv\AAPL.csv


C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_df = stock_df.append(price_df.loc[:,['Date','CODE','Adj Close']],sort=False)
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecat

./csv\AMZN.csv


C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append meth

./csv\BND.csv


...

C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append meth

./csv\SPY.csv


C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append meth

./csv\USM.csv


C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
C:\Users\moons\AppData\Local\Temp\ipykernel_7472\3800721193.py:37: FutureWarning: The frame.append meth

In [61]:
month_last_df.head()

,Date,CODE,1M_RET
18,2010-01-29,AAPL,0.897435
37,2010-02-26,AAPL,1.050789
60,2010-03-31,AAPL,1.124456
81,2010-04-30,AAPL,1.106454
101,2010-05-28,AAPL,0.964445


In [62]:
month_last_df.loc[18,:]

,Date,CODE,1M_RET
18,2010-01-29,AAPL,0.897435
18,2010-01-29,AMZN,0.936595
18,2010-01-29,BND,1.011692
18,2010-01-29,GDX,0.85349
18,2010-01-29,GLD,0.965027
18,2010-01-29,MSFT,0.910501
18,2010-01-29,SLV,0.922809
18,2010-01-29,SPY,0.947586
18,2010-01-29,USM,0.870922


In [63]:
stock_df.loc[0,:]

,Date,CODE,Adj Close
0,2010-01-04,AAPL,26.782711
0,2010-01-04,AMZN,133.899994
0,2010-01-04,BND,60.611969
0,2010-01-04,GDX,44.908779
0,2010-01-04,GLD,109.800003
0,2010-11-18,GM,27.027195
0,2010-01-04,MSFT,24.525019
0,2010-01-04,SLV,17.23
0,2010-01-04,SPY,93.675278
0,2010-01-04,USM,36.015179


In [64]:
month_last_df[(month_last_df['Date'] == '2010-01-29') | (month_last_df['Date'] == '2010-02-26')]

,Date,CODE,1M_RET
18,2010-01-29,AAPL,0.897435
37,2010-02-26,AAPL,1.050789
18,2010-01-29,AMZN,0.936595
37,2010-02-26,AMZN,0.996046
18,2010-01-29,BND,1.011692
37,2010-02-26,BND,1.003909
18,2010-01-29,GDX,0.85349
37,2010-02-26,GDX,1.022124
18,2010-01-29,GLD,0.965027
37,2010-02-26,GLD,1.009968


In [65]:
month_ret_df = month_last_df.pivot('Date','CODE','1M_RET').copy()
month_ret_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.897435,0.936595,1.011692,0.85349,0.965027,NaN,0.910501,0.922809,0.947586,0.870922
2010-02-26,1.050789,0.996046,1.003909,1.022124,1.009968,NaN,1.013871,0.982274,1.015404,1.004938
2010-03-31,1.124456,1.090172,0.996728,0.994848,0.995614,NaN,1.009304,1.063935,1.049976,1.117472
2010-04-30,1.106454,1.040134,1.011023,1.088577,1.046254,NaN,1.047325,1.041049,1.008574,0.996684
2010-05-28,0.964445,0.912503,1.011821,0.999599,1.027218,NaN,0.839806,0.978827,0.908766,0.968954
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,1.044274,1.008363,1.001883,0.985379,0.995904,1.018051,1.094653,0.980563,1.031919,0.817688
2019-03-29,1.085615,1.065214,1.022547,1.03318,1.001067,0.947842,1.048076,0.998592,1.011783,0.974942
2019-04-30,1.04931,1.061917,1.003719,0.951275,0.997285,1.031515,1.097295,0.990813,1.028653,1.035968


In [66]:
# 2단계. 상대모멘텀 수익률로 filtering 하기.
month_ret_df = month_last_df.pivot('Date','CODE','1M_RET').copy()
# 투자종목 선택할 rank 
month_ret_df = month_ret_df.rank(axis=1, ascending=False, method="max", pct=True) 
# 상위 40%에 드는 종목들만 Signal list.
month_ret_df = month_ret_df.where( month_ret_df < 0.4 , np.nan)
month_ret_df.fillna(0,inplace=True)
month_ret_df[month_ret_df != 0] = 1
stock_codes = list(stock_df['CODE'].unique())

In [67]:
month_ret_df.head(10)

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2010-02-26,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2010-03-31,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2010-04-30,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2010-05-28,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2010-06-30,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2010-07-30,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2010-08-31,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2010-09-30,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [68]:
# 3단계. signal list로 trading + positioning
sig_dict = dict()
for date in month_ret_df.index:
    # 신호가 포착된 종목코드만 읽어온다.
    ticker_list = list(month_ret_df.loc[date,month_ret_df.loc[date,:] >= 1.0].index)
    # 날짜별 종목코드 저장
    sig_dict[date] = ticker_list
stock_c_matrix = stock_df.pivot('Date','CODE','Adj Close').copy()
book = create_trade_book(stock_c_matrix, list(stock_df['CODE'].unique()))
book.head()

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM,...,p GM,r GM,p MSFT,r MSFT,p SLV,r SLV,p SPY,r SPY,p USM,r USM
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,26.782711,133.899994,60.611969,44.908779,109.800003,NaN,24.525019,17.23,93.675278,36.015179,...,,,,,,,,,,
2010-01-05,26.82901,134.690002,60.789135,45.341774,109.699997,NaN,24.532942,17.51,93.923241,35.998024,...,,,,,,,,,,
2010-01-06,26.40226,132.25,60.766037,46.443077,111.510002,NaN,24.382378,17.860001,93.989357,35.680672,...,,,,,,,,,,
2010-01-07,26.35346,130.0,60.719822,46.217175,110.82,NaN,24.128809,17.889999,94.386139,35.208931,...,,,,,,,,,,
2010-01-08,26.528664,133.520004,60.78141,46.913723,111.370003,NaN,24.295214,18.15,94.700218,34.651424,...,,,,,,,,,,


In [70]:
# 3단계. signal list로 trading + positioning
sig_dict = dict()
for date in month_ret_df.index:
    # 신호가 포착된 종목코드만 읽어온다.
    ticker_list = list(month_ret_df.loc[date,month_ret_df.loc[date,:] >= 1.0].index)
    # 날짜별 종목코드 저장
    sig_dict[date] = ticker_list
stock_c_matrix = stock_df.pivot('Date','CODE','Adj Close').copy()
book = create_trade_book(stock_c_matrix, list(stock_df['CODE'].unique()))

# positioning
for date,values in sig_dict.items():
    for stock in values:
        book.loc[date,'p '+ stock] = 'ready ' + stock
        
# 3-2  tradings
book = tradings(book, stock_codes)

# 4 단게. 수익률 계산하기.
multi_returns(book, stock_codes)

AAPL
AMZN
BND
GDX
GLD
GM
MSFT
SLV
SPY
USM
개별 청산일 :  2010-03-01  종목코드 :  BND long 진입가격 :  61.280487  |  long 청산가격 :  61.585163  | return: 0.5 %
개별 청산일 :  2010-03-01  종목코드 :  GLD long 진입가격 :  108.349998  |  long 청산가격 :  109.43  | return: 1.0 %
개별 청산일 :  2010-04-01  종목코드 :  GDX long 진입가격 :  42.019035  |  long 청산가격 :  43.675705  | return: 3.94 %
개별 청산일 :  2010-04-01  종목코드 :  SPY long 진입가격 :  90.145805  |  long 청산가격 :  97.770996  | return: 8.46 %
개별 청산일 :  2010-05-03  종목코드 :  AMZN long 진입가격 :  131.809998  |  long 청산가격 :  137.490005  | return: 4.31 %
개별 청산일 :  2010-05-03  종목코드 :  USM long 진입가격 :  36.212452  |  long 청산가격 :  36.744232  | return: 1.47 %
개별 청산일 :  2010-06-01  종목코드 :  AAPL long 진입가격 :  26.154476  |  long 청산가격 :  32.642082  | return: 24.8 %
개별 청산일 :  2010-06-01  종목코드 :  MSFT long 진입가격 :  24.56805  |  long 청산가격 :  20.70437  | return: -15.73 %
개별 청산일 :  2010-08-02  종목코드 :  BND long 진입가격 :  62.79007  |  long 청산가격 :  64.18457  | return: 2.22 %
개별 청산일 :  2010-08-02  종목코드 :  GDX long 진입

In [71]:
book.loc['2012-01-27':'2012-03-01',['AAPL','p AAPL','r AAPL']]

CODE,AAPL,p AAPL,r AAPL
Date,,,
2012-01-27,55.975765,buy AAPL,
2012-01-30,56.69286,buy AAPL,
2012-01-31,57.127102,ready AAPL,
2012-02-01,57.090816,buy AAPL,
2012-02-02,56.956921,buy AAPL,
2012-02-03,57.52758,buy AAPL,
2012-02-06,58.064457,buy AAPL,
2012-02-07,58.67268,buy AAPL,
2012-02-08,59.655083,buy AAPL,
